In [92]:
import joblib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import os
import re


In [93]:
def preprocess(text):
    # Substituir caracteres não alfabéticos por espaços em branco
    text = re.sub(r'[^a-zA-Zá-úÁ-Ú]', ' ', text)

    # Converter para minúsculas e dividir em palavras
    tokens = text.lower().split()

    return ' '.join(tokens)

In [94]:
def load_lyrics():
    return joblib.load('lyrics_vector.joblib')


In [95]:
def create_tfidf_matrix(lyrics):
    preprocessed_lyrics = [preprocess(text) for text in lyrics]
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(preprocessed_lyrics)
    return tfidf_matrix, vectorizer


In [114]:
def perform_query(query, vectorizer, tfidf_matrix, lyrics):
    preprocessed_query = preprocess(query)
    print(f"\nConsulta: {preprocessed_query}")
    query_vector = vectorizer.transform([preprocessed_query])

    # Calcular similaridade
    cosine_similarities = linear_kernel(query_vector, tfidf_matrix).flatten()

    # Contar o número de ocorrências da sequência na consulta
    sequence_count = preprocessed_query.count(' ')
    
    if sequence_count == 0:
        sequence_count = 1

    # Adicionar um fator de prioridade para palavras em sequência
    sequence_priority = 100.0  # Ajuste conforme necessário
    for i in range(len(lyrics)):
        document = lyrics[i]
        occurrences = document.lower().count(preprocessed_query)
        cosine_similarities[i] += sequence_priority * occurrences / sequence_count

    document_scores = list(enumerate(cosine_similarities))
    sorted_documents = sorted(document_scores, key=lambda x: x[1], reverse=True)

    # Imprimir resultados
    print("\nResultados:")
    for index, score in sorted_documents[:5]:
        print(
            f"Arquivo: {os.listdir('C:/Users/miguel/Documents/UFV/6 PERIODO/GRADI/musics-txt')[index]}, Pontuação: {score}")


In [97]:

# Carregar vetor de letras
print("Carregando letras...")
lyrics = load_lyrics()

# Criar matriz TF-IDF
print("Criando matriz TF-IDF...")
tfidf_matrix, vectorizer = create_tfidf_matrix(lyrics)
print(f"Dimensões da matriz TF-IDF: {tfidf_matrix.shape}")

Carregando letras...
Criando matriz TF-IDF...
Dimensões da matriz TF-IDF: (379679, 544933)


In [115]:
def main():
    # Menu de inserção de consulta
    while True:
        query = input("\nInsira sua consulta (ou 'exit' para sair): ")
        if query.lower() == 'exit':
            break
        perform_query(query, vectorizer, tfidf_matrix, lyrics)


if __name__ == "__main__":
    main()


Consulta: borboletas

Resultados:
Arquivo: adriana-partimpim - As Borboletas.txt, Pontuação: 1400.7454202424724
Arquivo: amado-batista - Borboletas.txt, Pontuação: 1300.8895286722866
Arquivo: elba-ramalho - A Dança Das Borboletas.txt, Pontuação: 500.3572833968989
Arquivo: ze-ramalho - A Dança Das Borboletas.txt, Pontuação: 500.32132190166357
Arquivo: palavra-cantada - A primavera da lagarta.txt, Pontuação: 500.16328012887465

Consulta: broboletas sempre voltam

Resultados:
Arquivo: ary-barroso - Quem Me Compreende.txt, Pontuação: 0.4145190162974804
Arquivo: anjos-anonimos - Mercúrio.txt, Pontuação: 0.3724580622081762
Arquivo: especial-de-ano-todo - Maio.txt, Pontuação: 0.3698479049854334
Arquivo: leno-e-lilian - Dias Iguais.txt, Pontuação: 0.362181531864885
Arquivo: sandy - Pés Cansados.txt, Pontuação: 0.30163262226088566

Consulta: ô mamae áfrica ô mamae áfrica

Resultados:
Arquivo: edson-gomes - Meus Direitos.txt, Pontuação: 0.7988048274134205
Arquivo: ponto-de-equilibrio - Moviment

KeyboardInterrupt: 